# Road Segmentation Project


In [ ]:
# Imports
import math
import os
import re
import cv2
import torch
import numpy as np
import parameters as params
import utils
import trainer
from processing import augment
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
from glob import glob
from random import sample
from PIL import Image
from torch import nn
from train import train
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from utils.datasets import ImageDataset
#from utils.dataset import ImageDataset, load_all_data
from utils.losses import DiceBCELoss
from utils import utils 
from models.cgan import Discriminator, GeneratorLoss, DiscriminatorLoss
from models.unet import UNet

In [ ]:
from transformer_seg import SETRModel, Vit

In [ ]:
# Loading data
device = 'cuda' if torch.cuda.is_available() else 'cpu'
images_org = utils.load_images(os.path.join(params.ROOT_PATH, 'training', 'images'), False)
masks_org = utils.load_images(os.path.join(params.ROOT_PATH, 'training', 'groundtruth'), True)

In [ ]:
## Uptraining
import sys
import wandb
from pathlib import Path
from tqdm import tqdm

CHECKPOINT_PATH = Path("checkpoints")
DATA_PATH = Path("data")


In [ ]:
model2 = SETRModel(patch_size=(32, 32), 
                    in_channels=3, 
                    out_channels=1, 
                    hidden_size=1024, 
                    sample_rate=5,
                    num_hidden_layers=1, 
                    num_attention_heads=16, 
                    decode_features=[512, 256, 128, 64])

In [ ]:
model2 = model2.to(device)

In [ ]:
t3 = torch.rand(1, 3, 384, 384).to(device)
print("input: " + str(t3.shape))

print("output: " + str(model2(t3).shape))

In [ ]:
print(model2(t3))

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 0)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)

model = SETRModel(patch_size=(32, 32), 
                    in_channels=3, 
                    out_channels=1, 
                    hidden_size=1024, 
                    sample_rate=5,
                    num_hidden_layers=1, 
                    num_attention_heads=16, 
                    decode_features=[512, 256, 128, 64])

model = model.to(device)
loss_fn = torch.nn.BCEWithLogitsLoss()
metric_fns = {'acc': trainer.accuracy_fn}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
#scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 100, None, 0, metric_fns)

In [ ]:
# Creating the submission on the test set
test_path = os.path.join(params.ROOT_PATH, 'training', 'images')
test_filenames = (glob(test_path + '/*.png'))
test_images = utils.load_all_from_path(test_path)
test_images2 = utils.load_all_from_path(test_path)
batch_size = test_images.shape[0]
size = test_images.shape[1:3]
# we also need to resize the test images. This might not be the best ideas depending on their spatial resolution.
test_images = np.stack([cv2.resize(img, dsize=(384, 384)) for img in test_images], 0)
test_images = test_images[:, :, :, :3]
test_images = utils.np_to_tensor(np.moveaxis(test_images, -1, 1), device)
test_pred = [model(t).detach().cpu().numpy() for t in test_images.unsqueeze(1)]
test_pred = np.concatenate(test_pred, 0)
test_pred= np.moveaxis(test_pred, 1, -1)  # CHW to HWC
test_pred = np.stack([cv2.resize(img, dsize=size) for img in test_pred], 0)

In [ ]:
utils.create_submission("test", "images",'setr_150_epoch.csv', model, device)

In [ ]:
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 50, None, 0, metric_fns)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 4, 1)
plt.title('Original Image')
plt.imshow(test_images2[0])
plt.subplot(1, 4, 2)
plt.title('Ilk Output')
plt.imshow(test_pred[0])
plt.subplot(1, 4, 3)
plt.title('LM Output')
plt.show()

In [ ]:
# ENsemble of Gans
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 0)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)

generator = UNet({'channels':[3,64,128,256,512,1024], 'activation': 'RELU'}).to(device)
discriminator = Discriminator(4).to(device)
generator_loss = GeneratorLoss()
discriminator_loss = DiscriminatorLoss()
metric_fns = {'acc': trainer.accuracy_fn, 'patch_acc': trainer.patch_accuracy_fn}
g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
trainer.train_pix2pix(train_dataloader, val_dataloader, generator, discriminator, generator_loss, discriminator_loss, metric_fns, g_optimizer, d_optimizer, 30)

In [ ]:
generators = []
generators.append(generator)

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.nn.functional.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice
        
class GeneratorLoss2(nn.Module):
    def __init__(self, alpha=100):
        super().__init__()
        self.alpha=alpha
        self.bce=DiceLoss()
        self.l1=nn.L1Loss()

    def forward(self, fake, real, fake_pred):
        fake_target = torch.ones_like(fake_pred)
        loss = self.bce(fake_pred, fake_target) + self.alpha* self.l1(fake, real)
        return loss

In [ ]:
# ENsemble of Gans
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 1)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)

generator = UNet({'channels':[3,64,128,256,512,1024], 'activation': 'RELU'}).to(device)
discriminator = Discriminator(4).to(device)
generator_loss = GeneratorLoss2()
discriminator_loss = DiscriminatorLoss()
metric_fns = {'acc': trainer.accuracy_fn, 'patch_acc': trainer.patch_accuracy_fn}
g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
trainer.train_pix2pix(train_dataloader, val_dataloader, generator, discriminator, generator_loss, discriminator_loss, metric_fns, g_optimizer, d_optimizer, 30)

In [ ]:
generators.append(generator)

In [ ]:
# Creating the submission on the test set
test_path = os.path.join(params.ROOT_PATH, 'test', 'images')
test_filenames = (glob(test_path + '/*.png'))
test_images = utils.load_all_from_path(test_path)
test_images2 = utils.load_all_from_path(test_path)
batch_size = test_images.shape[0]
size = test_images.shape[1:3]
# we also need to resize the test images. This might not be the best ideas depending on their spatial resolution.
test_images = np.stack([cv2.resize(img, dsize=(384, 384)) for img in test_images], 0)
test_images = test_images[:, :, :, :3]
test_images = utils.np_to_tensor(np.moveaxis(test_images, -1, 1), device)
test_pred = [generators[1](t).detach().cpu().numpy() for t in test_images.unsqueeze(1)]
test_pred = np.concatenate(test_pred, 0)
test_pred= np.moveaxis(test_pred, 1, -1)  # CHW to HWC
test_pred = np.stack([cv2.resize(img, dsize=size) for img in test_pred], 0)  # resize to original shape

In [ ]:
import pydensecrf.densecrf as dcrf
import numpy as np
from pydensecrf.utils import create_pairwise_gaussian, create_pairwise_bilateral, unary_from_softmax

def apply_dense_crf(model_pred, image, num_classes=2, iterations=10, sxy_gaussian=(3, 3), compat_gaussian=3):
    """
    Apply DenseCRF to the probabilities of an image.

    :param probabilities: The probability map of shape (num_classes, height, width)
    :param image: The original image of shape (height, width, channels)
    :param num_classes: Number of classes (default: 2 for binary classification)
    :param iterations: Number of iterations for CRF inference
    :param sxy_gaussian: Spatial kernel size for Gaussian kernel
    :param compat_gaussian: Compatibility for Gaussian kernel
    :return: Refined predictions
    """
    height, width = image.shape[:2]
    
    #probabilities = 1 / (1 + np.exp(-model_pred))
    probabilities_2d = np.zeros((2, height, width), dtype=np.float32)
    probabilities_2d[0, :, :] = 1 - model_pred
    probabilities_2d[1, :, :] = model_pred

    d = dcrf.DenseCRF2D(width, height, num_classes)

    # The unary potential is negative log probability
    unary = -np.log(probabilities_2d)
    unary = unary.reshape((num_classes, -1))
    d.setUnaryEnergy(unary)

    # Add pairwise Gaussian
    d.addPairwiseGaussian(sxy=sxy_gaussian, compat=compat_gaussian, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)

    image_uint8 = (image * 255).astype(np.uint8) if image.dtype == np.float32 else image.astype(np.uint8)

    # Add pairwise Bilateral
    d.addPairwiseBilateral(sxy=(80, 80), srgb=(13, 13, 13), rgbim=image_uint8, compat=10, kernel=dcrf.DIAG_KERNEL, normalization=dcrf.NORMALIZE_SYMMETRIC)

    # Perform inference
    Q = d.inference(iterations)
    result = np.argmax(Q, axis=0).reshape((height, width))

    return result

In [ ]:
test_images2 = test_images2[:,:,:,:3]
output = []
for i, pred in enumerate(test_pred):
    crf_result = apply_dense_crf(test_pred[i], test_images2[i], num_classes=2)
    output.append(crf_result)

In [ ]:
test_pred[0]

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 4, 1)
plt.title('Original Image')
plt.imshow(test_images2[0])
plt.subplot(1, 4, 2)
plt.title('Ilk Output')
plt.imshow(test_pred[0])
plt.subplot(1, 4, 3)
plt.title('LM Output')
plt.imshow(output[0])
plt.show()

In [ ]:
model4 = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model4 = model4.to(device)

checkpoint = torch.load('checkpoints/neat-serenity-95/epoch_15.pt')
model4.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
# Creating the submission on the test set
test_path = os.path.join(params.ROOT_PATH, 'test', 'images')
test_filenames = (glob(test_path + '/*.png'))
test_images = utils.load_all_from_path(test_path)
test_images2 = utils.load_all_from_path(test_path)
batch_size = test_images.shape[0]
size = test_images.shape[1:3]
# we also need to resize the test images. This might not be the best ideas depending on their spatial resolution.
test_images = np.stack([cv2.resize(img, dsize=(384, 384)) for img in test_images], 0)
test_images = test_images[:, :, :, :3]
test_images = utils.np_to_tensor(np.moveaxis(test_images, -1, 1), device)
test_pred = [model4(t).detach().cpu().numpy() for t in test_images.unsqueeze(1)]
test_pred = np.concatenate(test_pred, 0)
test_pred= np.moveaxis(test_pred, 1, -1)  # CHW to HWC
test_pred = np.stack([cv2.resize(img, dsize=size) for img in test_pred], 0)  # resize to original shape

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 4, 1)
plt.title('Original Image')
plt.imshow(test_images2[0])
plt.subplot(1, 4, 2)
plt.title('Ilk Output')
plt.imshow(test_pred[0])
plt.subplot(1, 4, 3)
plt.title('LM Output')
plt.imshow(output[0])
plt.show()

In [ ]:
# Creating the submission on the test set
test_path = os.path.join(params.ROOT_PATH, 'test', 'images')
test_filenames = (glob(test_path + '/*.png'))
test_images = utils.load_all_from_path(test_path)
test_images2 = utils.load_all_from_path(test_path)
batch_size = test_images.shape[0]
size = test_images.shape[1:3]
# we also need to resize the test images. This might not be the best ideas depending on their spatial resolution.
test_images = np.stack([cv2.resize(img, dsize=(384, 384)) for img in test_images], 0)
test_images = test_images[:, :, :, :3]
test_images = utils.np_to_tensor(np.moveaxis(test_images, -1, 1), device)

In [ ]:
model1 = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model1 = model1.to(device)
model2 = smp.Unet(
    encoder_name="efficientnet-b4",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model2 = model2.to(device)

model3 = smp.Unet(
    encoder_name="vgg19",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model3 = model3.to(device)

In [ ]:
checkpoint = torch.load('checkpoints/neat-serenity-95/epoch_15.pt')
model1.load_state_dict(checkpoint['model_state_dict'])

checkpoint = torch.load('checkpoints/sage-shadow-94/epoch_15.pt')
model2.load_state_dict(checkpoint['model_state_dict'])

checkpoint = torch.load('checkpoints/electric-water-93/epoch_15.pt')
model3.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
test_pred1 = [model1(t).detach().cpu().numpy() for t in test_images.unsqueeze(1)]
test_pred1 = np.concatenate(test_pred1, 0)
#print(test_pred.shape)
test_pred1= np.moveaxis(test_pred1, 1, -1)

test_pred2 = [model2(t).detach().cpu().numpy() for t in test_images.unsqueeze(1)]
test_pred2 = np.concatenate(test_pred2, 0)
#print(test_pred.shape)
test_pred2= np.moveaxis(test_pred2, 1, -1)

test_pred3 = [model3(t).detach().cpu().numpy() for t in test_images.unsqueeze(1)]
test_pred3 = np.concatenate(test_pred3, 0)
#print(test_pred.shape)
test_pred3= np.moveaxis(test_pred3, 1, -1)

In [ ]:
# Creating the submission on the test set
tra_images = utils.load_images(os.path.join(params.ROOT_PATH, 'training', 'images'), False)
gra_images = utils.load_images(os.path.join(params.ROOT_PATH, 'training', 'groundtruth'), True)

# we also need to resize the test images. This might not be the best ideas depending on their spatial resolution.
tra_images = np.stack([cv2.resize(img, dsize=(384, 384)) for img in tra_images], 0)
gra_images = np.stack([cv2.resize(img, dsize=(384, 384)) for img in gra_images], 0)
tra_images = tra_images[:, :, :, :3]
tra_images = utils.np_to_tensor(np.moveaxis(tra_images, -1, 1), device)
gra_images = utils.np_to_tensor(np.moveaxis(gra_images, -1, 1), device)

In [ ]:
tra_pred1 = [model1(t).detach().cpu().numpy() for t in test_images.unsqueeze(1)]
tra_pred1 = np.concatenate(tra_pred1, 0)
#print(test_pred.shape)
tra_pred1= np.moveaxis(tra_pred1, 1, -1)

tra_pred2 = [model2(t).detach().cpu().numpy() for t in test_images.unsqueeze(1)]
tra_pred2 = np.concatenate(tra_pred2, 0)
#print(test_pred.shape)
tra_pred2= np.moveaxis(tra_pred2, 1, -1)

tra_pred3 = [model3(t).detach().cpu().numpy() for t in test_images.unsqueeze(1)]
tra_pred3 = np.concatenate(tra_pred3, 0)
#print(test_pred.shape)
tra_pred3= np.moveaxis(tra_pred3, 1, -1)

In [ ]:
tra_pred1.shape

In [ ]:
# Stacking
from sklearn.linear_model import LogisticRegression

# Combine base model predictions
train_meta_features = np.stack((tra_pred1, tra_pred2, tra_pred3), axis=1)
test_meta_features = np.stack((test_pred1, test_pred2, test_pred3), axis=1)

# Train logistic regression as meta-model
meta_model = LogisticRegression()
meta_model.fit(train_meta_features, gra_images)

In [ ]:
train_meta_features.shape

In [ ]:
meta_test_preds = meta_model.predict(test_meta_features)

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 1)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)
    
model = smp.Unet(
    encoder_name="vgg19",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model = model.to(device)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
#scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 40, None, 0, metric_fns)

In [ ]:
model_backs = []
model_backs.append(model)

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 1)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)
    
model = smp.Unet(
    encoder_name="efficientnet-b4",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model = model.to(device)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
#scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 40, None, 0, metric_fns)

In [ ]:
model_backs.append(model)

In [ ]:
model4 = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model4 = model4.to(device)

checkpoint = torch.load('checkpoints/lively-surf-85/epoch_40.pt')
model4.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
model_backs.append(model4)

In [ ]:
models = []
for model_b in model_backs:
    train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

    images_aug, masks_aug = augment.augment_data(train_images, train_masks, 2)
    
    images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
    masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)
    
    val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
    val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)
    
    # reshape the image to simplify the handling of skip connections and maxpooling
    train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
    val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))
    
        
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)
        
    loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
    metric_fns = {'acc': trainer.accuracy_fn,
    'f1_score': trainer.f1_score_fn}
    optimizer = torch.optim.Adam(model_b.parameters(), lr=1e-5)
    #scheduler = ReduceLROnPlateau(optimizer)
    train(model_b, optimizer, train_dataloader, val_dataloader, loss_fn, 15, None, 0, metric_fns)
    models.append(model_b)

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 1)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)
    
model = smp.Unet(
    encoder_name="efficientnet-b5",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model = model.to(device)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
#scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 40, None, 0, metric_fns)
model_backs.append(model)

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 1)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)
    
model = smp.Unet(
    encoder_name="resnext50_32x4d",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model = model.to(device)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
#scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 40, None, 0, metric_fns)
model_backs.append(model)

In [ ]:
for model_b in model_backs[-2:]:
    train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

    images_aug, masks_aug = augment.augment_data(train_images, train_masks, 2)
    
    images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
    masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)
    
    val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
    val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)
    
    # reshape the image to simplify the handling of skip connections and maxpooling
    train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
    val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))
    
        
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)
        
    loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
    metric_fns = {'acc': trainer.accuracy_fn,
    'f1_score': trainer.f1_score_fn}
    optimizer = torch.optim.Adam(model_b.parameters(), lr=1e-5)
    #scheduler = ReduceLROnPlateau(optimizer)
    train(model_b, optimizer, train_dataloader, val_dataloader, loss_fn, 15, None, 0, metric_fns)
    models.append(model_b)

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 1)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)
    
model = smp.Unet(
    encoder_name="vgg19",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model = model.to(device)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
#scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 40, None, 0, metric_fns)

In [ ]:
model4 = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model4 = model4.to(device)

checkpoint = torch.load('checkpoints/lively-surf-85/epoch_40.pt')
model4.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
#scheduler = ReduceLROnPlateau(optimizer)
train2(model4, optimizer, images_org, masks_org, loss_fn, 15, None, 0, metric_fns)

In [ ]:
model1 = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model1 = model1.to(device)
model2 = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model2 = model2.to(device)

model3 = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model3 = model3.to(device)

In [ ]:
checkpoint = torch.load('checkpoints/divine-thunder-64/epoch_15.pt')
model1.load_state_dict(checkpoint['model_state_dict'])

checkpoint = torch.load('checkpoints/worldly-snow-83/epoch_15.pt')
model2.load_state_dict(checkpoint['model_state_dict'])

checkpoint = torch.load('checkpoints/dutiful-donkey-84/epoch_15.pt')
model3.load_state_dict(checkpoint['model_state_dict'])



In [ ]:
utils.create_submission("test", "images",'worldly_snow.csv', model2, device)

In [ ]:
len(models)

In [ ]:
# Making prediction
test_path = os.path.join(params.ROOT_PATH, "test", "images")
test_filenames = (glob(test_path + '/*.png'))
test_images = utils.load_all_from_path(test_path)
batch_size = test_images.shape[0]
size = test_images.shape[1:3]

test_images = np.stack([cv2.resize(img, dsize=(384, 384)) for img in test_images], 0)
test_images = test_images[:, :, :, :3]
test_images = utils.np_to_tensor(np.moveaxis(test_images, -1, 1), device)

preds = utils.ensemble_predict(models, [1,1,1,1,1], test_images)

test_pred = np.stack([cv2.resize(img, dsize=size) for img in preds], 0)  # resize to original shape
# now compute labels
test_pred = test_pred.reshape((-1, size[0] // params.PATCH_SIZE, params.PATCH_SIZE, size[0] // params.PATCH_SIZE, params.PATCH_SIZE))
test_pred = np.moveaxis(test_pred, 2, 3)
test_pred = np.round(np.mean(test_pred, (-1, -2)) > params.CUTOFF)
with open("ensemble_staged_training_5_diff.csv", 'w') as f:
        f.write('id,prediction\n')
        for fn, patch_array in zip(sorted(test_filenames), test_pred):
            img_number = int(re.search(r"satimage_(\d+)", fn).group(1))
            for i in range(patch_array.shape[0]):
                for j in range(patch_array.shape[1]):
                    f.write("{:03d}_{}_{},{}\n".format(img_number, j*params.PATCH_SIZE, i*params.PATCH_SIZE, int(patch_array[i, j])))
                    

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 1)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=True)
    
model2 = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model2 = model2.to(device)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model2.parameters(), lr=1e-4)
#scheduler = ReduceLROnPlateau(optimizer)
train(model2, optimizer, train_dataloader, val_dataloader, loss_fn, 20, None, 0, metric_fns)

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 2)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=True)
    
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model2.parameters(), lr=1e-5)
#scheduler = ReduceLROnPlateau(optimizer)
train(model2, optimizer, train_dataloader, val_dataloader, loss_fn, 15, None, 0, metric_fns)

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 3)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=True)
    
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model2.parameters(), lr=1e-5)
#scheduler = ReduceLROnPlateau(optimizer)
train(model2, optimizer, train_dataloader, val_dataloader, loss_fn, 15, None, 0, metric_fns)

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 1)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)
    
model = smp.Unet(
    encoder_name="resnet101",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model = model.to(device)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
#scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 20, None, 0, metric_fns)

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 2)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)

loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
#scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 20, None, 0, metric_fns)

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 1)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=3, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=3, shuffle=True)
    
model = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model = model.to(device)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
#scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 40, None, 0, metric_fns)

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 2)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=True)
    
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,
'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
#scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 15, None, 0, metric_fns)

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
    )

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 2)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=True)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
#scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 20, None, 0, metric_fns)

In [ ]:
import ttach as tta

transforms = tta.Compose(
    [
        tta.HorizontalFlip(),
        tta.VerticalFlip(),
        tta.Rotate90([0,90])
    ]
)

tta_models = []
for i in range(5):
    tta_models.append(tta.SegmentationTTAWrapper(models[i], transforms))

preds = utils.ensemble_predict(tta_models, [1,1,1,1,1], test_images)
test_pred = np.stack([cv2.resize(img, dsize=size) for img in preds], 0)  # resize to original shape
# now compute labels
test_pred = test_pred.reshape((-1, size[0] // params.PATCH_SIZE, params.PATCH_SIZE, size[0] // params.PATCH_SIZE, params.PATCH_SIZE))
test_pred = np.moveaxis(test_pred, 2, 3)
test_pred = np.round(np.mean(test_pred, (-1, -2)) > params.CUTOFF)
with open("ensemble_resunet_50_5_aug_tta.csv", 'w') as f:
        f.write('id,prediction\n')
        for fn, patch_array in zip(sorted(test_filenames), test_pred):
            img_number = int(re.search(r"satimage_(\d+)", fn).group(1))
            for i in range(patch_array.shape[0]):
                for j in range(patch_array.shape[1]):
                    f.write("{:03d}_{}_{},{}\n".format(img_number, j*params.PATCH_SIZE, i*params.PATCH_SIZE, int(patch_array[i, j])))

In [ ]:
models.append(model)

In [ ]:
models.append(model)

In [ ]:
utils.create_submission("test", "images",'resnet_trained_further.csv', model, device)

In [ ]:
from torch.optim.lr_scheduler import CosineAnnealingLR

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
)

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 2)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=True)
    

model = models[0].to(device)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 20, scheduler, 0, metric_fns)

In [ ]:
utils.create_submission("test", "images",'resnet_trained_even_further.csv', model, device)

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
train_images, val_images, train_masks, val_masks = train_test_split(
        images_org, masks_org, test_size=0.1, random_state=42, shuffle=True
)

images_aug, masks_aug = augment.augment_data(train_images, train_masks, 2)

images_aug = np.stack([img/255.0 for img in images_aug]).astype(np.float32)
masks_aug = np.stack([mask/255.0 for mask in masks_aug]).astype(np.float32)

val_images = np.stack([img/255.0 for img in val_images]).astype(np.float32)
val_masks = np.stack([mask/255.0 for mask in val_masks]).astype(np.float32)

# reshape the image to simplify the handling of skip connections and maxpooling
train_dataset = ImageDataset(images_aug, masks_aug, device, use_patches=False, resize_to=(384, 384))
val_dataset = ImageDataset(val_images, val_masks, device, use_patches=False, resize_to=(384, 384))

    
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=2, shuffle=True)
    

model = models[1].to(device)
loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
metric_fns = {'acc': trainer.accuracy_fn,'f1_score': trainer.f1_score_fn}
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
scheduler = ReduceLROnPlateau(optimizer)
train(model, optimizer, train_dataloader, val_dataloader, loss_fn, 15, scheduler, 0, metric_fns)